In [ ]:
!pip uninstall -y transformers adapters peft accelerate huggingface_hub datasets tokenizers -q
!pip install -q transformers==4.46.3 tokenizers==0.20.3 huggingface_hub==0.26.2 datasets==3.1.0 accelerate==1.1.1
!pip install -q adapters==1.0.1 peft==0.13.2 evaluate rouge_score wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 129.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 131.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.2/333.2 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 20.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.50.0 requires huggingface-hub<2.0,>=0.33.5, but you have huggingface-hub 0.26.2 which is incompatible.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.9.0 which is incompatible.
diffusers 0.36.0 requires huggingface-hub<2.0,>=0.34.0, b

# NLP Project

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/nlp-project')

In [ ]:
import wandb
import os

wandb.login()

os.environ["WANDB_PROJECT"] = "nlp-project-peft"

wandb: Currently logged in as: tombik (tombik-warsaw-university-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Prepare data

In [ ]:
from datasets import DatasetDict
from transformers import AutoTokenizer

from src.data import load_ai_ga

ds_ai_ga = load_ai_ga()
print(ds_ai_ga)

DatasetDict({
    train: Dataset({
        features: ['title', 'abstract', 'label'],
        num_rows: 22930
    })
    validation: Dataset({
        features: ['title', 'abstract', 'label'],
        num_rows: 2866
    })
    test: Dataset({
        features: ['title', 'abstract', 'label'],
        num_rows: 2866
    })
})


In [ ]:
from src.data import load_xlsum

ds_es = load_xlsum("spanish")
ds_ne = load_xlsum("nepali")

print(ds_es)
print(ds_ne)

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text'],
        num_rows: 38110
    })
    test: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text'],
        num_rows: 4763
    })
    validation: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text'],
        num_rows: 4763
    })
})
DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text'],
        num_rows: 5808
    })
    test: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text'],
        num_rows: 725
    })
    validation: Dataset({
        features: ['id', 'url', 'title', 'summary', 'text'],
        num_rows: 725
    })
})


Analiza zbiorów danych

In [ ]:
from src.utils import average_length_ai_ga, average_length_xlsum, show_sample

In [ ]:
print("=== ANALIZA AI-GA ===")
average_length_ai_ga(ds_ai_ga)
show_sample(ds_ai_ga, "AI-GA")

=== ANALIZA AI-GA ===
Średnia długość abstraktu w AI-GA (w słowach): 189.81

=== SAMPLE FROM AI-GA ===
title: Scientific Opinion on the public health hazards to be covered by inspection of meat from sheep and goats
abstract: 

This study explores the potential public health hazards associated with meat from sheep and goats and provides an opinion on the inspection procedures that should be implemented. It is well known that traditional livestock can cause several food-borne diseases due to their origin, production and p...
label: 1


In [ ]:
print("\n=== ANALIZA XL-SUM (Spanish) ===")
average_length_xlsum(ds_es, "XL-Sum Spanish")
show_sample(ds_es, "XL-Sum Spanish")


=== ANALIZA XL-SUM (Spanish) ===

XL-Sum Spanish:
Średnia długość artykułu: 832.78 słów
Średnia długość streszczenia: 27.93 słów

=== SAMPLE FROM XL-SUM SPANISH ===
id: 140930_ultnot_siria_onu_comida_ch
url: https://www.bbc.com/mundo/ultimas_noticias/2014/09/140930_ultnot_siria_onu_comida_ch
title: ONU dice que peligran raciones de comida para Siria
summary: La directora de ayuda humanitaria de Naciones Unidas, Valerie Amos, advirtió que de no invertir más dinero, el Programa Mundial de Alimentos tendrá que detener sus operaciones en Siria en dos meses.
text: De no recibir más dinero, las raciones de la ONU en Siria se terminarán en dos meses. En un informe al Consejo de Seguridad de la ONU, Amos dijo que las raciones del PMA destinadas a los 4.000.000 de sirios ya han sido recortadas para poder llegar a la mayor cantidad de personas posible. Amos tambié...


In [ ]:
print("\n=== ANALIZA XL-SUM (Nepali) ===")
average_length_xlsum(ds_ne, "XL-Sum Nepali")
show_sample(ds_ne, "XL-Sum Nepali")


=== ANALIZA XL-SUM (Nepali) ===

XL-Sum Nepali:
Średnia długość artykułu: 376.84 słów
Średnia długość streszczenia: 19.82 słów

=== SAMPLE FROM XL-SUM NEPALI ===
id: news-55625366
url: https://www.bbc.com/nepali/news-55625366
title: कोरोना भाइरस महामारीः चीनले कसरी आवाज दबायो र कोभिड-१९ को इतिहासको पुनर्लेखन गर्‍यो
summary: गत वर्षको आरम्भमै चीन सरकारले दुईवटा चुनौती खेप्नुपरेको थियो। देशभरि ठूलो खतराका रूपमा देखिएको एउटा अज्ञात रोग अनि त्यहाँको अवस्थाबारे इन्टरनेटको माध्यमबाट विश्वभरि सूचना दिइरहेका मानिसहरूको आवाज शक्तिशाली चीनका ठूला चुनौती थिए।
text: तर सन् २०२० को अन्त्यसम्म सरकारद्वारा नियन्त्रित सञ्चारमाध्यमलाई गहिरिएर हेर्ने हो भने दुवै चुनौती अब नियन्त्रण आइसकेको जस्तो देखिन्छ। बीबीसीका केरी इलन र जाओयिन फेङले कसरी चीन सरकारले नकारात्मक समाचारलाई अङ्कुश लगायो अनि त्यो अवस्थामा कसरी केही नागरिकले त्यहाँको जानकारी प्रवाह गर्न सके तथा सरकारी स...


Baseline

In [ ]:
from src.training import baseline_bert_ai_ga, baseline_mt5_zero_shot

In [ ]:
bert_baseline_results = baseline_bert_ai_ga(ds_ai_ga)
bert_baseline_results

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/22930 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/2866 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro
1,0.007600,0.005850,0.998953,0.998953


{'eval_loss': 0.005734259262681007,
 'eval_accuracy': 0.9989532449406838,
 'eval_f1_macro': 0.9989532437937572,
 'eval_runtime': 37.6589,
 'eval_samples_per_second': 76.104,
 'eval_steps_per_second': 4.78,
 'epoch': 1.0}

In [ ]:
print("=== ZERO-SHOT SPANISH ===")
baseline_es = baseline_mt5_zero_shot(ds_es)

print("\n=== ZERO-SHOT NEPALI ===")
baseline_ne = baseline_mt5_zero_shot(ds_ne)

=== ZERO-SHOT SPANISH ===


tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]


Generating summaries: 100%|██████████| 25/25 [00:11<00:00,  2.11it/s]



=== ZERO-SHOT NEPALI ===


Generating summaries: 100%|██████████| 25/25 [00:19<00:00,  1.31it/s]


In [ ]:
print(baseline_es)

{'rouge1': np.float64(0.04182163400606013), 'rouge2': np.float64(0.008502547404655416), 'rougeL': np.float64(0.03637444926567859), 'rougeLsum': np.float64(0.03676534920723104)}


In [ ]:
print(baseline_ne)

{'rouge1': np.float64(0.0), 'rouge2': np.float64(0.0), 'rougeL': np.float64(0.0), 'rougeLsum': np.float64(0.0)}


## Konfiguracja Eksperymentów PEFT

In [ ]:
from peft import LoraConfig, PrefixTuningConfig, TaskType

# --- BERT (Klasyfikacja) ---

# 1. LoRA dla BERT
peft_config_bert_lora = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    target_modules=["query", "value"]
)

# 2. Prefix Tuning dla BERT
peft_config_bert_prefix = PrefixTuningConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    num_virtual_tokens=20
)

# --- mT5 (Seq2Seq / Streszczanie) ---

# 3. LoRA dla mT5
peft_config_mt5_lora = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    target_modules=["q", "v"]
)

# 4. Prefix Tuning dla mT5
peft_config_mt5_prefix = PrefixTuningConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    num_virtual_tokens=30
)

### Uruchamianie eksperymentów


In [ ]:
import src.training as training
import importlib

importlib.reload(training)

<module 'src.training' from '/content/drive/MyDrive/nlp-project/src/training.py'>

In [ ]:
from src.training import train_bert, train_mt5

### 1. Baseline

In [ ]:
wandb.init(
    project="nlp-project-peft",
    group="bert",
    name="full-fine-tuning",
)
try:
  print("=== TRENING BERT FULL FINE-TUNING ===")
  results_bert_full = train_bert(
      ds_ai_ga,
      peft_config=None,
      output_dir="./bert_full_output",
      num_epochs=3
  )
  print(results_bert_full)
finally:
  wandb.finish()

In [ ]:
wandb.init(
    project="nlp-project-peft",
    group="mt5-es",
    name="full-fine-tuning",
)
print("\n=== TRENING mT5 Spanish FULL FINE-TUNING ===")
try:
  results_mt5_es_full = train_mt5(
      ds_es,
      peft_config=None,
      output_dir="./mt5_es_full_output",
      num_epochs=3
  )
  print(results_mt5_es_full)
finally:
  wandb.finish()


=== TRENING mT5 Spanish FULL FINE-TUNING ===


/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:558: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.991000,2.706363,0.246200,0.067700,0.186200,0.186400,41.673700


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.991000,2.706363,0.246200,0.067700,0.186200,0.186400,41.673700


In [ ]:
wandb.init(
    project="nlp-project-peft",
    group="mt5-ne",
    name="full-fine-tuning-mt5-ne",
)
try:
  print("\n=== TRENING mT5 Nepali FULL FINE-TUNING ===")
  results_mt5_ne_full = train_mt5(
      ds_ne,
      peft_config=None,
      output_dir="./mt5_ne_full_output",
      num_epochs=3
  )
  print(results_mt5_ne_full)
finally:
  wandb.finish()

wandb: Detected [huggingface_hub.inference] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/



=== TRENING mT5 Nepali FULL FINE-TUNING ===


tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

...

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.696800,2.220122,0.000000,0.000000,0.000000,0.000000,43.052400
2,2.438500,2.076310,0.000000,0.000000,0.000000,0.000000,43.740700
3,2.166600,2.042662,0.000000,0.000000,0.000000,0.000000,44.422100


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


{'eval_loss': 2.237050771713257, 'eval_rouge1': 0.0, 'eval_rouge2': 0.0, 'eval_rougeL': 0.0, 'eval_rougeLsum': 0.0, 'eval_gen_len': 43.4731, 'eval_runtime': 113.433, 'eval_samples_per_second': 6.391, 'eval_steps_per_second': 0.802, 'epoch': 3.0}


eval/gen_len,▁▅█▃
eval/loss,▇▂▁█
eval/rouge1,▁▁▁▁
eval/rouge2,▁▁▁▁
eval/rougeL,▁▁▁▁
eval/rougeLsum,▁▁▁▁
eval/runtime,█▆▁▅
eval/samples_per_second,▁▃█▄
eval/steps_per_second,▁▃█▄
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
+4,...


In [ ]:
print("\n=== TRENING mT5 Spanish (Downsampled) FULL FINE-TUNING ===")
ds_es_downsampled = DatasetDict({
    "train": ds_es["train"].shuffle(seed=42).select(range(len(ds_ne["train"]))),
    "validation": ds_es["validation"],
    "test": ds_es["test"]
})
wandb.init(
    project="nlp-project-peft",
    group="mt5-es",
    name="full-fine-tuning-mt5-es-low",
)
try:
  results_mt5_es_downsampled = train_mt5(
      ds_es_downsampled,
      peft_config=None,
      output_dir="./mt5_es_downsampled_output",
      num_epochs=3
  )
  print(results_mt5_es_downsampled)
finally:
  wandb.finish()


=== TRENING mT5 Spanish (Downsampled) FULL FINE-TUNING ===


/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:558: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/5808 [00:00<?, ? examples/s]

Map:   0%|          | 0/4763 [00:00<?, ? examples/s]

Map:   0%|          | 0/4763 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,4.006000,3.643897,0.158600,0.016200,0.130000,0.130000,56.763200
2,3.335600,3.101066,0.202900,0.039200,0.159900,0.159900,49.096400
3,3.053400,2.960583,0.226300,0.051000,0.171800,0.171800,43.293500


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


{'eval_loss': 2.956714153289795, 'eval_rouge1': 0.2275, 'eval_rouge2': 0.0501, 'eval_rougeL': 0.1722, 'eval_rougeLsum': 0.1723, 'eval_gen_len': 43.3149, 'eval_runtime': 742.8018, 'eval_samples_per_second': 6.412, 'eval_steps_per_second': 0.802, 'epoch': 3.0}


eval/gen_len,█▄▁▁
eval/loss,█▂▁▁
eval/rouge1,▁▆██
eval/rouge2,▁▆██
eval/rougeL,▁▆██
eval/rougeLsum,▁▆██
eval/runtime,█▇▁▁
eval/samples_per_second,▁▂██
eval/steps_per_second,▁▂██
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
+4,...


### 2. Bert - klasyfikacja

BERT + LoRA

In [ ]:
wandb.init(
    project="nlp-project-peft",
    group="bert",
    name="LoRa-r16",
)
try:
  print("\n=== TRENING BERT LoRA ===")
  results_bert_lora = train_bert(
      ds_ai_ga,
      peft_config=peft_config_bert_lora,
      output_dir="./bert_lora_output",
      num_epochs=3
  )
  print(results_bert_lora)
finally:
  wandb.finish()

BERT + Prefix Tuning

In [ ]:
wandb.init(
    project="nlp-project-peft",
    group="bert",
    name="prefix-bert",
)
try:
  print("\n=== TRENING BERT Prefix Tuning ===")
  results_bert_prefix = train_bert(
      ds_ai_ga,
      peft_config=peft_config_bert_prefix,
      output_dir="./bert_prefix_output",
      num_epochs=3
  )
  print(results_bert_prefix)
finally:
  wandb.finish()


=== TRENING BERT Prefix Tuning ===


Map:   0%|          | 0/22930 [00:00<?, ? examples/s]

Map:   0%|          | 0/2866 [00:00<?, ? examples/s]

Map:   0%|          | 0/2866 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 370,178 || all params: 109,853,956 || trainable%: 0.3370


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro
1,0.652900,0.624396,0.741103,0.737495
2,0.592200,0.575989,0.771458,0.769143
3,0.593500,0.559702,0.782624,0.780944


{'eval_loss': 0.563853919506073, 'eval_accuracy': 0.771109560362875, 'eval_f1_macro': 0.7685049879702619, 'eval_runtime': 40.0964, 'eval_samples_per_second': 71.478, 'eval_steps_per_second': 4.489, 'epoch': 3.0}


eval/accuracy,▁▆█▆
eval/f1_macro,▁▆█▆
eval/loss,█▃▁▁
eval/runtime,▁▇▄█
eval/samples_per_second,█▂▅▁
eval/steps_per_second,█▂▆▁
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇█████
train/global_step,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇█████
train/grad_norm,██▃▁▃▃▃▇▁▇▃▄▁▃█▁▁▁▅▄▆▁▁▁▁▂▁▁▁▁▃▂▂▁▂▄▂▃▂▁
train/learning_rate,███▇▇▇▇▇▆▆▆▆▆▆▆▆▅▅▅▄▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▁▁
+1,...


### 3. MT5 - Generacja

mT5 + LoRA

In [ ]:
wandb.init(
    project="nlp-project-peft",
    group="mt5-es",
    name="LoRa-r16",
)
try:
  print("\n=== TRENING mT5 Spanish LoRA ===")
  results_mt5_es_lora = train_mt5(
      ds_es,
      peft_config=peft_config_mt5_lora,
      output_dir="./mt5_es_lora_output",
      num_epochs=3
  )
  print(results_mt5_es_lora)
finally:
  wandb.finish()

mT5 + Prefix Tuning

In [ ]:
wandb.init(
    project="nlp-project-peft",
    group="mt5-es",
    name="prefix-mt5-es",
)
try:
  print("\n=== TRENING mT5 Spanish Prefix Tuning ===")
  results_mt5_es_prefix = train_mt5(
      ds_es,
      peft_config=peft_config_mt5_prefix,
      output_dir="./mt5_es_prefix_output",
      num_epochs=3
  )
  print(results_mt5_es_prefix)
finally:
  wandb.finish()


=== TRENING mT5 Spanish Prefix Tuning ===


/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:558: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


trainable params: 184,320 || all params: 300,361,088 || trainable%: 0.0614


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,6.830500,4.195394,0.107400,0.013600,0.096200,0.096100,63.000000
2,6.074600,3.963020,0.138700,0.022000,0.120200,0.120100,51.500900
3,5.932600,3.870645,0.136800,0.021600,0.115600,0.115600,26.540400


{'eval_loss': 3.9615838527679443, 'eval_rouge1': 0.1384, 'eval_rouge2': 0.0224, 'eval_rougeL': 0.1197, 'eval_rougeLsum': 0.1197, 'eval_gen_len': 51.6009, 'eval_runtime': 756.4543, 'eval_samples_per_second': 6.296, 'eval_steps_per_second': 0.788, 'epoch': 3.0}


eval/gen_len,█▆▁▆
eval/loss,█▃▁▃
eval/rouge1,▁███
eval/rouge2,▁█▇█
eval/rougeL,▁█▇█
eval/rougeLsum,▁█▇█
eval/runtime,██▁█
eval/samples_per_second,▁▁█▁
eval/steps_per_second,▁▁█▁
train/epoch,▁▁▁▁▁▂▂▂▂▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇████
+4,...


mT5 + LoRA (Nepali)

In [ ]:
wandb.init(
    project="nlp-project-peft",
    group="mt5-ne",
    name="LoRa-r16",
)
try:
  print("\n=== TRENING mT5 Nepali LoRA ===")
  results_mt5_ne_lora = train_mt5(
      ds_ne,
      peft_config=peft_config_mt5_lora,
      output_dir="./mt5_ne_lora_output",
      num_epochs=3
  )
  print(results_mt5_ne_lora)
finally:
  wandb.finish()

mT5 + Prefix Tuning (Nepali)

In [ ]:
wandb.init(
    project="nlp-project-peft",
    group="mt5-ne",
    name="prefix",
)
try:
  print("\n=== TRENING mT5 Nepali Prefix Tuning ===")
  results_mt5_ne_prefix = train_mt5(
      ds_ne,
      peft_config=peft_config_mt5_prefix,
      output_dir="./mt5_ne_prefix_output",
      num_epochs=3
  )
  print(results_mt5_ne_prefix)
finally:
  wandb.finish()

### 4. Adaptery

BERT + Adapter (Houlsby)

In [ ]:
wandb.init(
    project="nlp-project-peft",
    group="bert",
    name="adapter-houlsby",
)
try:
  print("\n=== TRENING BERT Adapter (Houlsby) ===")
  results_bert_adapter = train_bert(
      ds_ai_ga,
      adapter_config="houlsby",
      output_dir="./bert_adapter_output",
      num_epochs=3
  )
  print(results_bert_adapter)
finally:
  wandb.finish()

mT5 + Adapter (Pfeiffer)

In [ ]:
wandb.init(
    project="nlp-project-peft",
    group="mt5-es",
    name="adapter-pfeiffer",
)
try:
  print("\n=== TRENING mT5 Spanish Adapter (Pfeiffer) ===")
  results_mt5_es_adapter = train_mt5(
      ds_es,
      adapter_config="pfeiffer",
      output_dir="./mt5_es_adapter_output",
      num_epochs=3
  )
  print(results_mt5_es_adapter)
finally:
  wandb.finish()

mT5 + Adapter (Nepali)

In [ ]:
wandb.init(
    project="nlp-project-peft",
    group="mt5-ne",
    name="adapter-pfeiffer",
)
try:
  print("\n=== TRENING mT5 Nepali Adapter (Pfeiffer) ===")
  results_mt5_ne_adapter = train_mt5(
      ds_ne,
      adapter_config="pfeiffer",
      output_dir="./mt5_ne_adapter_output",
      num_epochs=3
  )
  print(results_mt5_ne_adapter)
finally:
  wandb.finish()

EXTRA (Low-Data)

In [ ]:
ds_es_downsampled = DatasetDict({
    "train": ds_es["train"].shuffle(seed=42).select(range(len(ds_ne["train"]))),
    "validation": ds_es["validation"],
    "test": ds_es["test"]
})

In [ ]:
wandb.init(
    project="nlp-project-peft",
    group="mt5-es-low",
    name="mt5-es-low-lora",
)
try:
  print("\n=== TRENING mT5 Nepali LoRA ===")
  results_mt5_ne_lora = train_mt5(
      ds_es_downsampled,
      peft_config=peft_config_mt5_lora,
      output_dir="./mt5_ne_lora_output",
      num_epochs=3
  )
  print(results_mt5_ne_lora)
finally:
  wandb.finish()

In [ ]:
wandb.init(
    project="nlp-project-peft",
    group="mt5-es-low",
    name="mt5-es-low-prefix",
)
try:
  print("\n=== TRENING mT5 Nepali Prefix Tuning ===")
  results_mt5_ne_prefix = train_mt5(
      ds_es_downsampled,
      peft_config=peft_config_mt5_prefix,
      output_dir="./mt5_ne_prefix_output",
      num_epochs=3
  )
  print(results_mt5_ne_prefix)
finally:
  wandb.finish()

In [ ]:
wandb.init(
    project="nlp-project-peft",
    group="mt5-ne",
    name="mt5-es-low-adapter-pfeiffer",
)
try:
  results_mt5_ne_adapter = train_mt5(
      ds_es_downsampled,
      adapter_config="pfeiffer",
      output_dir="./mt5_ne_adapter_output",
      num_epochs=3
  )
  print(results_mt5_ne_adapter)
finally:
  wandb.finish()